## 분장 분리(마침표를 기준으로 문장 분리) 및 전처리(정규표현식으로 진행)

In [37]:
import re
import json
from collections import Counter
from importlib.resources import contents

import pandas as pd
import pymysql
import re
from ekonlpy.tag import Mecab

In [38]:
#문장분리기
def sen_split(text):
    return re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s', str(text))

In [39]:
#정규 표현식으로 필요없는 부분 제거
def tokenize_korean_text(text):
    patterns = [
        r'([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)',  # E-mail
        r'(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+',  # URL
        r'<[^>]*>',  # HTML tags
        r'([ㄱ-ㅎㅏ-ㅣ]+)',  # Korean consonants and vowels
        r'([가-힣A-Za-z·\d~\-\.]{2,}(로|길)+)',  # Address patterns
        r'([가-힣A-Za-z·\d~\-\.]+(읍|동|번지|시|구)+)',
        r'([가-힣A-Za-z]+(구)\s*[가-힣A-Za-z]+(동))',
        r'([가-힣a-zA-Z\d]+(아파트|빌라|빌딩|마을))',
        r'\n',  # 줄바꿈 제거
        r'[`~!@#$%^&*()_|+\-=?;:,.<>\{\}\[\]\\\/]',  # Special characters
        r'[0-9]'  # Numbers
    ]
    
    for pattern in patterns:
        text = re.sub(pattern, "", text)
    return text

In [40]:
#각 문서에 대해 문장 분리 및 정규표현식 진행
def start_split_sentences(text):
    sentences = sen_split(text)
    return [tokenize_korean_text(sentence) for sentence in sentences]

In [41]:
df = pd.read_csv("Result_5.csv", header=None, names=["id", "document_type", "date", "title", "media", "sentences", "sentiment_score_market", "reg_date", "content_text"])
df["content_text"] = df["content_text"].apply(json.loads)
df["sentences"] = df["sentences"].apply(json.loads)
df.dtypes

id                         int64
document_type             object
date                      object
title                     object
media                     object
sentences                 object
sentiment_score_market     int64
reg_date                  object
content_text              object
dtype: object

In [45]:
# #데이터 베이스 접근 및 전처리 진행
# class DatabaseManager:
#     def __init__(self, host, port, user, password, db_name, charset="utf8"):
#         # 데이터베이스 연결 설정
#         self.connection = pymysql.connect(
#             host=host,
#             port=port,
#             user=user,
#             passwd=password,
#             db=db_name,
#             charset=charset
#         )
# 
#     def fetch_sentences(self, batch_size=1000):
#         """주어진 쿼리로 데이터를 가져오는 함수"""
#         with self.connection.cursor(pymysql.cursors.DictCursor) as cursor:
#             last_id = 0
#             while True:
#                 sql = """
#                 SELECT id, sentences
#                 FROM text_analysis 
#                 WHERE document_type = 'analyst_report' AND id > %s
#                 ORDER BY id
#                 LIMIT %s
#                 """
#                 cursor.execute(sql, (last_id, batch_size))
#                 results = cursor.fetchall()
# 
#                 if not results:
#                     break
# 
#                 # for test
#                 if int(results[-1]["id"]) > 2000:
#                     break
# 
#                 for row in results:
#                     yield row
# 
#                 last_id = results[-1]['id']
# 
#     def process_data(self, batch_size=1000):
#         processed_data = []
#         for row in self.fetch_sentences(batch_size):
#             try:
#                 d = json.loads(row['sentences'])
#                 text = d.get('keyword', '')
#                 sentences = start_split_sentences(text)
#                 processed_data.append({'id': row['id'], 'keyword': sentences})
#                 if len(processed_data) % 100 == 0:
#                     print(f"Processed {len(processed_data)} documents")
#             except json.JSONDecodeError as e:
#                 print(f"Error processing document {row['id']}: {e}")
#         return processed_data
# 
#     def close(self):
#         """데이터베이스 연결 닫기"""
#         self.connection.close()
# 
# def main():
#     # 데이터베이스 연결 정보
#     db_manager = DatabaseManager(
#         host='paper-project-1.cne82okq8qfy.us-east-2.rds.amazonaws.com',
#         port=3306,
#         user="choi",
#         password="UbcFp1C!7^shuxKYv4Q$",
#         db_name="PAPER_PROJECT"
#     )
# 
#     try:
#         #SQL 쿼리 실행 및 데이터 가져오기
#         processed_data = db_manager.process_data()
#         print(f"Total processed documents: {len(processed_data)}")
#     finally:
#         db_manager.close()
# 
#     return processed_data
# 
# main()


OperationalError: (2003, "Can't connect to MySQL server on 'paper-project-1.cne82okq8qfy.us-east-2.rds.amazonaws.com' (timed out)")

## 품사태깅 
- 이전에 ekonlpy를 설치해야 함.
- git clone https://github.com/entelecheia/eKoNLPy.git
- cd eKoNLPy
- pip install .
반드시 konlpy와 Mecab이 설치되어 있는지 ekonlpy 설치 전 확인 해야 함.

In [42]:
#문장에서 명사만 추출
def extract_nouns_from_sentences(contents):
    mecab = Mecab()
    res = []
    stopwords = ['NNG', 'NNP','VAX','VV','VCN']  #일반 명사(NNG), 고유 명사(NNP)만 추출

    for text in contents:  #contents는 문장의 리스트 ([[문장1], [문장2], ...])
        tokens = mecab.pos(text)  #형태소 분석
        tokens = mecab.replace_synonyms(tokens)  #동의어 치환
        tokens = mecab.lemmatize(tokens)  #원형 복원
        
        print(tokens)
        
        # 명사만 추출
        nouns = [t_0 for t_0, t_1, *_ in tokens if t_1 in stopwords]
        res.append(nouns)  # 문장별로 명사만 추출하여 결과 리스트에 추가

    return res


In [43]:
def generate_ngrams(text, n):
    words = text.split()
    return [' '.join(words[i:i+n]) for i in range(len(words) - n + 1)]

def process_ngrams_to_dict(sentences):
    ngram_counts = {i: Counter() for i in range(1, 6)}  # 1-gram부터 5-gram까지의 Counter 객체
    
    for sentence in sentences:
        for n in range(1, 6):
            ngrams = generate_ngrams(sentence, n)
            ngram_counts[n].update(ngrams)
    
    return ngram_counts

def update_ngram_dict(row, sentences_column, ngram_column, total_counts):
    new_ngrams = process_ngrams_to_dict(row[sentences_column])
    
    # 기존 딕셔너리가 없으면 새로 생성
    if not isinstance(row[ngram_column], dict):
        row[ngram_column] = {}
    
    # 각 n에 대해 기존 딕셔너리 업데이트 및 total_counts 업데이트
    for n, gram_counter in new_ngrams.items():
        n_gram = f"{n}-gram"
        if n_gram not in row[ngram_column]:
            row[ngram_column][n_gram] = {}
        for gram, count in gram_counter.items():
            row[ngram_column][n_gram][gram] = row[ngram_column][n_gram].get(gram, 0) + count
            total_counts[n_gram][gram] += count
    
    return row

def process_dataframe(df, sentences_column, ngram_column):
    total_counts = {f"{i}-gram": Counter() for i in range(1, 6)}
    df = df.apply(lambda row: update_ngram_dict(row, sentences_column, ngram_column, total_counts), axis=1)
    return df, total_counts

# N-gram 생성, dict 컬럼 업데이트 및 전체 카운트 계산
df, total_counts = process_dataframe(df, 'content_text', 'sentences')


KeyboardInterrupt: 

In [7]:
df.head()

,id,document_type,date,title,media,sentences,sentiment_score_market,reg_date,content_text
0,7734,analyst_report,2008-04-01 00:00:00,대우증권 월간 채권투자 _4월_,대우증권,{'keyword': ['2008_04 ¿ù°£Ã¤±ÇÅõÀÚ Ã¤±Ç½ÃÀåÀ...,0,2024-09-02 00:00:00,[2008_04 \n¿ù°£Ã¤±ÇÅõÀÚ\nÃ¤±Ç½ÃÀåÀü¸Á\n±ÝÀ¶½Ã...
1,7732,analyst_report,2008-04-07 00:00:00,시간차 공격,대우증권,{'keyword': ['Fixed Income Weekly 2008. 4. 7....,0,2024-09-02 00:00:00,[Fixed Income Weekly \n2008. 4. 7. #918 \nFix...
2,7733,analyst_report,2008-04-07 00:00:00,돌아온 박스권,대우증권,{'keyword': ['Fixed Income Weekly 2008. 4. 7....,0,2024-09-02 00:00:00,[Fixed Income Weekly \n2008. 4. 7. #918 \nFix...
3,7731,analyst_report,2008-04-14 00:00:00,총재의 변신은 무죄다__,대우증권,{'keyword': [' 4월 금통위는 정부-한은간 policy mix ...,0,2024-09-02 00:00:00,[ \n \n 4월 금통위는 정부-한은간 policy mix 합의의 반영일 수도 ...
4,7730,analyst_report,2008-04-21 00:00:00,항상 열려는 있는 가능성,대우증권,{'keyword': [' 경기 전망은 비교적 분명 … 인플레 명분이 시점...,0,2024-09-02 00:00:00,[ \n \n 경기 전망은 비교적 분명 … 인플레 명분이 시점 결정할 것 \n현 ...


In [8]:
total_counts

{'1-gram': Counter({',': 112980,
          '.': 101197,
          '자료:': 99000,
          '-': 57167,
          '수': 53640,
          '및': 53101,
          '금리': 49468,
          '0': 48385,
          '회사채': 43524,
          ')': 43001,
          '미국': 41760,
          '스프레드': 39526,
          '0.0': 35718,
          '리서치센터': 35522,
          '이': 34347,
          '것으로': 33200,
          '3Y': 32690,
          '대한': 31622,
          'AAA': 29589,
          '있다.': 25675,
          'AA-': 25040,
          ':': 24905,
          '추이': 23270,
          '그림': 23096,
          '기준금리': 21045,
          'Bloomberg,': 20759,
          '국고채': 20707,
          '전망': 20453,
          '주:': 19986,
          '하나금융투자': 19314,
          '5Y': 19240,
          '국채': 19122,
          '1': 18895,
          '등': 18811,
          '2': 18772,
          '3': 18632,
          'A+': 18321,
          '있는': 18098,
          '10': 18015,
          '/': 17926,
          '본': 17797,
          '이후': 17788,
          

In [9]:
def analyze_low_count_ngrams(total_counts, threshold=15):
    low_count_analysis = {}
    for n, counter in total_counts.items():
        low_count_ngrams = {gram: count for gram, count in counter.items() if count < threshold}
        low_count_analysis[n] = {
            'count': len(low_count_ngrams),
            'examples': list(low_count_ngrams.items())[:10]  # 예시로 10개만 보여줍니다
        }
    return low_count_analysis

low_counts = analyze_low_count_ngrams(total_counts)
low_counts

{'1-gram': {'count': 950164,
  'examples': [('2008_04', 1),
   ('¿ù°£Ã¤±ÇÅõÀÚ', 2),
   ('Ã¤±Ç½ÃÀåÀü¸Á', 2),
   ('±ÝÀ¶½ÃÀåÂ÷Æ®ºÏ', 1),
   ('Óßëë°úùÚÍ£', 1),
   ('對應과', 4),
   ('限界', 4),
   ('다방면의', 2),
   ('그것은,', 6),
   ('모면케', 2)]},
 '2-gram': {'count': 5107746,
  'examples': [('2008_04 ¿ù°£Ã¤±ÇÅõÀÚ', 1),
   ('¿ù°£Ã¤±ÇÅõÀÚ Ã¤±Ç½ÃÀåÀü¸Á', 2),
   ('Ã¤±Ç½ÃÀåÀü¸Á ±ÝÀ¶½ÃÀåÂ÷Æ®ºÏ', 1),
   ('±ÝÀ¶½ÃÀåÂ÷Æ®ºÏ Óßëë°úùÚÍ£', 1),
   ('4월 2', 5),
   ('2 對應과', 1),
   ('對應과 限界', 4),
   ('限界 미국에서는', 1),
   ('미국에서는 여전히,', 1),
   ('여전히, 연준은', 1)]},
 '3-gram': {'count': 7956823,
  'examples': [('2008_04 ¿ù°£Ã¤±ÇÅõÀÚ Ã¤±Ç½ÃÀåÀü¸Á', 1),
   ('¿ù°£Ã¤±ÇÅõÀÚ Ã¤±Ç½ÃÀåÀü¸Á ±ÝÀ¶½ÃÀåÂ÷Æ®ºÏ', 1),
   ('Ã¤±Ç½ÃÀåÀü¸Á ±ÝÀ¶½ÃÀåÂ÷Æ®ºÏ Óßëë°úùÚÍ£', 1),
   ('월간채권투자 4월 2', 2),
   ('4월 2 對應과', 1),
   ('2 對應과 限界', 1),
   ('對應과 限界 미국에서는', 1),
   ('限界 미국에서는 여전히,', 1),
   ('미국에서는 여전히, 연준은', 1),
   ('여전히, 연준은 물론', 1)]},
 '4-gram': {'count': 9139031,
  'examples': [('2008_04 ¿ù°£Ã¤±ÇÅõÀÚ Ã¤±Ç½ÃÀåÀü¸Á ±ÝÀ¶½ÃÀåÂ÷Æ®ºÏ', 1),
   ('¿ù°£Ã

In [10]:
def analyze_high_count_ngrams(total_counts, threshold=15):
    high_count_analysis = {}
    high_count_dict = {}
    
    for n, counter in total_counts.items():
        high_count_ngrams = {gram: count for gram, count in counter.items() if count >= threshold}
        high_count_analysis[n] = {
            'count': len(high_count_ngrams),
            'examples': list(high_count_ngrams.items())[:5]  # 예시로 5개만 보여줍니다
        }
        high_count_dict[n] = high_count_ngrams
    
    return high_count_analysis, high_count_dict

high_counts, total_counts = analyze_high_count_ngrams(total_counts)

In [11]:
high_counts

{'1-gram': {'count': 66170,
  'examples': [('월간채권투자', 495),
   ('4월', 6749),
   ('2', 18772),
   ('미국에서는', 188),
   ('여전히,', 15)]},
 '2-gram': {'count': 91646,
  'examples': [('월간채권투자 4월', 64),
   ('노력을 하고', 15),
   ('하고 있다.', 307),
   ('있다. 하지만', 421),
   ('해 줄', 26)]},
 '3-gram': {'count': 61194,
  'examples': [('없을 것 같다.', 23),
   ('수 있다고 본다.', 59),
   ('크지는 않을 것으로', 15),
   ('것으로 보고 있다.', 75),
   ('서철수 02-768-3501 cyberscs@bestez.com', 76)]},
 '4-gram': {'count': 50916,
  'examples': [('서철수 02-768-3501 cyberscs@bestez.com 윤여삼', 62),
   ('02-768-3501 cyberscs@bestez.com 윤여삼 02-768-4022', 62),
   ('cyberscs@bestez.com 윤여삼 02-768-4022 nersam@bestez.com', 22),
   ('윤여삼 02-768-4022 nersam@bestez.com 윤일광', 31),
   ('02-768-4022 nersam@bestez.com 윤일광 02-768-3058', 31)]},
 '5-gram': {'count': 44839,
  'examples': [('서철수 02-768-3501 cyberscs@bestez.com 윤여삼 02-768-4022', 62),
   ('02-768-3501 cyberscs@bestez.com 윤여삼 02-768-4022 nersam@bestez.com', 22),
   ('cyberscs@bestez.com 윤여삼 02-768-402

In [12]:
total_counts

{'1-gram': {'월간채권투자': 495,
  '4월': 6749,
  '2': 18772,
  '미국에서는': 188,
  '여전히,': 15,
  '연준은': 2393,
  '물론': 1877,
  '정부와': 377,
  '의회': 418,
  '역시': 5003,
  '신용경색': 212,
  '해소를': 88,
  '위해': 4846,
  '노력을': 228,
  '하고': 2424,
  '있다.': 25675,
  '하지만': 3222,
  '미': 14446,
  '금융시스템의': 51,
  '붕괴는': 19,
  '해': 4545,
  '줄': 1339,
  '수': 53640,
  '순환적': 64,
  '만들': 134,
  '수는': 1524,
  '없을': 1076,
  '것': 11640,
  '같다.': 781,
  '금융시장의': 673,
  '유동성이': 569,
  '가운데에서도': 31,
  '침체로': 89,
  '가는': 357,
  '상황이라면,': 18,
  '이는': 5447,
  '국내': 14045,
  '금리에도': 47,
  '하락': 9915,
  '있다고': 3081,
  '본다.': 1250,
  '이미': 3180,
  '정책이': 769,
  '진행중인': 56,
  '미국과': 1265,
  '달리': 1395,
  '국내에서는': 79,
  '향후': 10581,
  '정책의': 710,
  '방향성을': 462,
  '놓고': 304,
  '확연히': 74,
  '드러나고': 18,
  '있는': 18098,
  '상황이다.': 880,
  '금통위에서도': 73,
  '그다지': 140,
  '금리': 49468,
  '우호적': 546,
  '발언이': 478,
  '나오지': 103,
  '않을': 4636,
  '리스크를': 491,
  '경계하고': 40,
  '있는데,': 318,
  '시장': 11488,
  '충격이': 415,
  '크지는': 72,
  '것으로': 33200,

In [ ]:
# result = dict()
# 
# # 불러오기
# for content in contents:
#     content_id = content['id']
#     nouns = extract_nouns_from_sentences(content.get('keyword', []))
#     result[content_id] = nouns
#     # print(result[-1])  # 마지막 문장의 명사 리스트 출력


In [ ]:
df["sentences"] = df["sentences"].apply(lambda x: x["nouns"])

In [3]:
#불용어 처리
def remove_stopwords(result):
    stop_words = ['층','n','마산','가','를','을','점','관','월','일','층','텔','남','좌','리','중','정','미','종']
    filtered_words = []
    for word in result:
        if word not in stop_words:
            filtered_words.append(word)
    return filtered_words

#불러오기
all_tokens=[]
for i in range(len(result)):
    all_tokens.append(remove_stopwords(result[i], stop_words))
all_tokens

[['월간',
  '채권투자',
  '對應',
  '限界',
  '미국',
  'fed',
  '정부',
  '의회',
  '신용경색',
  '해소',
  '다방면',
  '노력'],
 ['스템', '붕괴', '모면', '경기', '침체'],
 ['장의', '보강', '가운데', '실물', '경기', '상황', '라면', '국내', '금리', '하락', '여건'],
 ['정책', '진행', '미국', '국내', '정책', '방향', '당국', '입장차', '상황'],
 ['금통위', '금리', '발언', '위험', '경계', '시장', '충격'],
 ['관건', '객관', '환경', '유리', '전개'],
 ['시간', '한은', '가능성', '우세'],
 ['신흥국',
  '신용위험',
  '둔화',
  '가운데',
  '한국',
  '외화',
  '조달',
  '비용',
  '자료',
  '서철수',
  '윤여',
  '윤일광',
  '對應',
  '限界'],
 ['동향'],
 ['채권투자', '환경'],
 ['요약',
  '전망',
  '차트',
  '국내',
  '금리',
  '동향',
  '국내',
  '금리',
  '주요',
  '스프레드',
  '채권',
  '수급',
  '채권',
  '수급장',
  '금리',
  '동향',
  '상품',
  '지표',
  '환율',
  '주식시장',
  '시장리스크',
  '인플레이션',
  '지표',
  '자금시장',
  '동향',
  '통화',
  '지표',
  '지표',
  '이슈',
  '스프레드',
  '전우',
  '월간',
  '채권투자'],
 ['동향',
  '강세',
  '흐름',
  '자체',
  '유지',
  '환율',
  '정책',
  '위험',
  '부각',
  '금리',
  '심화',
  '전반',
  '금리',
  '레벨',
  '가운데',
  '다양',
  '인해',
  '금리',
  '확대'],
 ['단기',
  '급락',
  '이후',
  '축소',
  '양상',
  '인플레이션

In [4]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora

#사전 구축
dictionary = corpora.Dictionary(all_tokens) # 토큰 단어와 gensim 내부 아이디 매칭
dictionary.filter_extremes(no_below=2, no_above=0.5) # 빈도 2이상 포함, 전체 50% 이상 단어 제거
corpus = [dictionary.doc2bow(token) for token in all_tokens] 

#LDA 모델 학습
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=3, random_state=42, passes=10)

#토픽 출력
print(lda_model.print_topics(num_words=5))

[(0, '0.053*"자료" + 0.035*"금리" + 0.025*"리서치센터" + 0.024*"국고채" + 0.022*"상승"'), (1, '0.029*"CDS" + 0.024*"정책" + 0.021*"fed" + 0.019*"상황" + 0.016*"자료"'), (2, '0.022*"한은" + 0.017*"정부" + 0.014*"금리" + 0.014*"가능성" + 0.014*"물가"')]


In [5]:
#시각화
import pyLDAvis.gensim

pyLDAvis.enable_notebook() # added
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics=False)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.160683  0.012009       1        1  36.870614
1      0.066537 -0.110841       2        1  29.750491
2      0.094146  0.098833       3        1  33.378895, topic_info=      Term       Freq      Total Category  logprob  loglift
49      자료  86.000000  86.000000  Default  30.0000  30.0000
339    CDS  32.000000  32.000000  Default  29.0000  29.0000
244  리서치센터  33.000000  33.000000  Default  28.0000  28.0000
44      한은  28.000000  28.000000  Default  27.0000  27.0000
85     국고채  32.000000  32.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
117     영향   7.827383  13.821646   Topic3  -5.0184   0.5286
28      정책  10.339668  38.718912   Topic3  -4.7401  -0.2231
87      인하   8.283861  18.208308   Topic3  -4.9617   0.3097
90      상승   8.052677  37.105377   Topic3  -4.9900  -0.4305
67      환율   7.454620  21.943867   Topic3  -5.0672   0.0176

[202 rows x 6 columns], token_table=      Topic      Freq Term
term                      
339       1  0.061031  CDS
339       2  0.915466  CDS
190       1  0.356009  GDP
190       3  0.623016  GDP
0         1  0.302644  fed
...     ...       ...  ...
75        3  0.199705   확대
67        1  0.637991   환율
67        3  0.318996   환율
263       1  0.981139  회사채
76        1  0.901366   흐름

[281 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])